In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [2]:
# Loading the datasets
train_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_Train.csv')
test_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_test.csv')
validation_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_Validation.csv')

In [3]:
def encode_cyclical_features(df, column, max_val):
    df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_val)
    df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_val)
    return df

In [4]:
def preprocess_data(df):
    df['local_time'] = pd.to_datetime(df['local_time'])
    
    for time_unit in ['Year', 'Month', 'Day', 'Hour']:
        df[time_unit] = getattr(df['local_time'].dt, time_unit.lower())
    
    # Cyclical time encoding
    df = encode_cyclical_features(df, 'Hour', 24) 
    df = encode_cyclical_features(df, 'Month', 12) 

    # Temperature interactions
    df['Temp_Hour_interaction'] = df['Average_Temp'] * df['Hour']
    
    df['Is_weekend'] = df['local_time'].dt.dayofweek >= 5 
    
    return df 


In [5]:
# Applying preprocessing to all datasets
train_df = preprocess_data(train_df)
validation_df = preprocess_data(validation_df)
test_df = preprocess_data(test_df)


In [6]:
print(train_df.columns)


Index(['local_time', 'Average_Temp', 'MW', 'Year', 'Month', 'Day', 'Hour',
       'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos',
       'Temp_Hour_interaction', 'Is_weekend'],
      dtype='object')


In [7]:
if 'Hour_sin' not in train_df.columns:
    print("Column 'Hour_sin' is missing from the DataFrame.")
else:
    print("Column 'Hour_sin' exists.")


Column 'Hour_sin' exists.


In [8]:
# Preparing the feature sets and target variables
def get_features_targets(df):
    feature_columns = [
        'Year', 'Month', 'Day', 'Hour', 'Average_Temp',
        'Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos',
        'Temp_Hour_interaction', 'Is_weekend'
    ]

    X = df[feature_columns]
    y = df['MW']
    return X, y


X_train, y_train = get_features_targets(train_df)
X_validation, y_validation = get_features_targets(validation_df)
X_test, y_test = get_features_targets(test_df)

In [9]:
# Training Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
from joblib import dump
# Save the model to a file
model_filename = '/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_linear_model_V2.joblib'
dump(linear_model, model_filename)

['/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_linear_model_V2.joblib']

In [10]:
# Evaluation Function

def calculate_metrics(actual, predicted, lower_bound=0, upper_bound=100, iqr_multiplier=1.5):
    # Excluding negative actual values if considered invalid
    valid_indices = actual > lower_bound
    actual = actual[valid_indices]
    predicted = predicted[valid_indices]

    # Calculate MAE and RMSE
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))

    # Thresholding for outlier exclusion based on IQR
    q1, q3 = np.percentile(actual, [25, 75])
    iqr = q3 - q1
    outlier_threshold_upper = q3 + (iqr * iqr_multiplier)
    outlier_threshold_lower = q1 - (iqr * iqr_multiplier)

    valid_indices_for_mape = (actual >= outlier_threshold_lower) & (actual <= outlier_threshold_upper)
    filtered_actual = actual[valid_indices_for_mape]
    filtered_predicted = predicted[valid_indices_for_mape]

    # Calculate Modified MAPE with capped at 100%
    if len(filtered_actual) > 0:
        percentage_errors = np.abs((filtered_predicted - filtered_actual) / filtered_actual) * 100
        percentage_errors = np.clip(percentage_errors, None, upper_bound)  # Cap percentage errors at upper_bound (100%)
        mape = np.mean(percentage_errors)
    else:
        mape = np.nan

    # Calculate sMAPE
    smape = 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

    return mae, mape, smape, rmse


In [11]:
from joblib import load
model_filename = '/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_linear_model_V2.joblib'
best_model_linear_loaded = load(model_filename)

In [12]:
# Predict and evaluate on validation set first for model tuning
predictions_validation = best_model_linear_loaded.predict(X_validation)
metrics_validation = calculate_metrics(y_validation, predictions_validation)
print("Validation Metrics (MAE, MAPE, sMAPE,RMSE):", metrics_validation)

Validation Metrics (MAE, MAPE, sMAPE,RMSE): (75.0344600157248, 85.83057887566707, 83.76433512725217, 83.81089126045917)


In [13]:
# Predict on the test set using the linear model
predictions_test_linear = best_model_linear_loaded.predict(X_test)
metrics_test_linear = calculate_metrics(y_test, predictions_test_linear)
print("Test Metrics (MAE, MAPE, sMAPE,RMSE):", metrics_test_linear)


Test Metrics (MAE, MAPE, sMAPE,RMSE): (81.23796174295089, 94.19657939640605, 87.71737284516794, 84.60999316255155)
